In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.linalg as lnlg
from scipy.linalg import qr

import copy
from IPython.display import display, Latex

# locals
from N2_SUSY_SYK import HamiltonianGenerator 

########## Macros ###########################
N_SAMPLES = 10
NP_RANDOM_SEED = 0
np.random.seed(NP_RANDOM_SEED)

# Physical constants
N = 6 # number of fermions
J = 100 # ~"energy scale" of couplings
N_DIM = 2**N # dimension of Hilbert space
Q_COUPLING = 3 # number of couplings per term

# Directories
N2_SUSY_DIR = os.path.join("Excel", "N2_SUSY_SYK")
RESULT_DIR = os.path.join(N2_SUSY_DIR, "Simulated Hamiltonians", f"N{N}_J{J}")


# 1. Define test matrix

It's always possible to put a Hermitian matrix (and in fact, any normal matrix) into block-diagonal form through unitary transformations (i.e. similarity transformations, preserving eigenvalues). Moreover, the Hamiltonians we work with are also Hermitian. So we might as well start with a sample Hamiltonian from the $\mathcal{N}=2$ supersymmetric SYK model.

An extra advantage is that we know these can be decomposed into fermionic and bosonic modes, i.e. a 2-block block-diagonal format (that is, if I understood Gustavo correctly at our last meeting, 9/01/2023). 

In [ ]:
h_generator = HamiltonianGenerator(N, J)
H = h_generator.make_H(0).toarray()

print(f"H hermitian: {lnlg.ishermitian(H)}")
print(f"H.shape: {H.shape}")

# 2. Define block-diagonalization algorithm

1. Find eigenvectors and eigenvalues

In [ ]:
ivals, ivecs = np.linalg.eigh(H)
print(f"ivecs.shape: {ivecs.shape}")

2. Orthonormalize eigenvectors using Graham-Schmidt procedure
3. Stack orthonormal eigenvectors to form unitary matrix, $U$

In [ ]:
def gram_schmidt(ivecs):
    (n, m) = ivecs.shape
    for i in range(m):
        q = ivecs[:, i] # i-th column of A
        for j in range(i):
            q = q - np.dot(ivecs[:, j], ivecs[:, i]) * ivecs[:, j]
        if np.array_equal(q, np.zeros(q.shape)):
            raise np.linalg.LinAlgError("The column vectors are not linearly independent")
    
        # normalize q
        q = q / np.sqrt(q@q)
        # write the vector back in the matrix
        ivecs[:, i] = q

U = copy.deepcopy(ivecs)
gram_schmidt(U)

# Check that they're indeed orthonormal

orthonormal = True
for i in range(N_DIM):
    for j in range(i, N_DIM):
        if i==j:
            inner_ij = U[:, i]@U[:, j]
            if not np.isclose(inner_ij, 1):
                orthonormal = False
                break
        else:
            inner_ij = U[:, i]@U[:, j]
            if not np.isclose(inner_ij, 0):
                orthonormal = False
                break

print(f"ivecs orthonormal: {orthonormal}")

4. $P = U^\dagger H U$ should be in block-diagonal form

In [ ]:
P = np.conjugate(np.transpose(U))@H@U

# Check that P is block diagonal

# First, round off small values to zero
cutoff = 1e-12 # <-- TODO: A rigorous way to determine a good floating-point cutoff (i.e. what is "close enough" to zero to be considered equivalently zero?)
               # For now, I'm just checking that max_zero_row (computed below) agrees with what I find by inspection (opening up P_mag in the variable explorer)

P_mag = np.abs(P)
P_mag[P_mag < cutoff] = 0

print(f"P_mag:\n{P_mag}")
P[P_mag < cutoff] = 0

5. Define the top-down function for block-diagonalization

In [ ]:
def jordan_normal(A):
    ivals, ivecs = np.linalg.eigh(A)
    gram_schmidt(ivecs)
    P = np.conjugate(np.transpose(ivecs))@A@ivecs
    return P, ivecs, ivals

P, ivecs_P, ivals_P = jordan_normal(H)

P_mag_round = np.abs(P)
P_mag_round[P_mag_round < cutoff] = 0


# 3. Extract the 2 blocks

In [ ]:
max_0_row = 0
for i in range(N_DIM):
    if np.all(P_mag_round[i, :] == 0):
        max_0_row = i
print(f"max_0_row: {max_0_row}")

P_upper = P[:max_0_row+1, :max_0_row+1]
P_lower = P[max_0_row+1:, max_0_row+1:]

print(f"P_upper.shape: {P_upper.shape}")
print(f"P_lower.shape: {P_lower.shape}")

# 4. For fun, let's apply this to the lower block once more

In [ ]:
P_P_lower, ivecs_lower, ivals_lower = jordan_normal(P_lower)
Q_mag = np.abs(Q)
Q_mag[Q_mag < cutoff] = 0
Q[Q_mag < cutoff] = 0
print(f"Q: \n{Q}")